<a href="https://colab.research.google.com/github/diegotluz/projects-colab/blob/main/My_Ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y pciutils
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
#Instalando o Ollama
!curl https://ollama.ai/install.sh | sh

In [ ]:
#@markdown **Connecting to Cloudflare...**

!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import os
import subprocess
import threading
import time
import socket
import urllib.request
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '127.0.0.1:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nOllama finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      cf_url = l[l.find("http"):]
      print("This is the URL to access Ollama:", cf_url, end='')

threading.Thread(target=iframe_thread, daemon=True, args=(11434,)).start()
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()
# !python main.py --dont-print-server

In [ ]:
!ollama pull codegemma:7b

In [ ]:
prompt="você é humano?"
url = 'http://localhost:11434/api/chat'
payload = {
    "model": "llama3:8b-text-q6_K",
    "temperature": 0.6,
    "stream": False,
    "messages": [
        {"role": "system", "content": "You are an AI assistant!"},
        {"role": "user", "content": prompt}
    ]
}

response = requests.post(url, json=payload)
message_str = response.content.decode('utf-8')
message_dict = json.loads(message_str)
print(message_dict['message']['content'])